In [1]:
import pandas as pd , os 
from sqlalchemy import create_engine, text as sql_text
from sqlalchemy.orm import Session
from dotenv import load_dotenv
import plotly.express as px

load_dotenv(r'C:\Users\SheharyarMonnoo\myCode\2 - DataPipeLine Automation\Analytics\.env')

True

In [2]:
data = {
    '2023-01': [163864.16, 112937.16, 198976.75, 202631.42, -446.12],
    '2023-02': [171001.34, 105629.91, 180039.57, 184808.94, 402.62],
    '2023-03': [203794.01, 188633.06, 258066.90, 255200.32, 255.73],
    '2023-04': [170459.30, 155811.22, 208092.32, 219693.40, 236.00],
    '2023-05': [138728.68, 129409.28, 175060.89, 176492.31, 3164.99]
}

index = ['Arlington', 'Denton', 'Keller', 'Frisco', 'Unapplied Payment']

df = pd.DataFrame(data, index=index).reset_index()
df.melt('index').groupby(['index'],as_index=False)['value'].sum().to_csv('KareoPmts.csv')

In [3]:
def connectDB():    

    # SQL Server connection parameters
    SERVER = str(os.getenv('SERVER'))
    DATABASE = str(os.getenv('DATABASE'))
    USERNAME_DB = str(os.getenv('USERNAME_DB'))
    PASSWORD_DB = str(os.getenv('PASSWORD_DB'))
    CONNECTION_TYPE = 'pymssql'
    
    connection_str = f'mssql+{CONNECTION_TYPE}://{USERNAME_DB}:{PASSWORD_DB}@{SERVER}/{DATABASE}'
    engine = create_engine(connection_str)
    
    return engine

engine = connectDB()

In [4]:
query = f"""

SELECT *
  FROM [dbo].[NextGenAppts]
  
  where servicePeriod >= '2023-01'  
  and servicerPeriod <= '2024-11'
  and patientName not like '%TEST%'
  and patientName not like '%Peter Pan%'
  and patientName not like '%TRAINING%'


"""

In [5]:
df = pd.read_sql(sql=sql_text(query), con=engine.connect())

servicePeriods = df['servicePeriod'].unique()
servicePeriods.sort()

locations = df['locationName'].unique()
locations

ProgrammingError: (pymssql._pymssql.ProgrammingError) (207, b"Invalid column name 'servicerPeriod'.DB-Lib error message 20018, severity 16:\nGeneral SQL Server error: Check messages from the SQL Server\n")
[SQL: 

SELECT *
  FROM [dbo].[NextGenAppts]
  
  where servicePeriod >= '2023-01'  
  and servicerPeriod <= '2024-11'
  and patientName not like '%TEST%'
  and patientName not like '%Peter Pan%'
  and patientName not like '%TRAINING%'


]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
df['Quarter'] = pd.to_datetime(df['servicePeriod']).dt.to_period('Q')

In [ ]:
data = {
    'Location': ['Arlington', 'Denton', 'Frisco', 'Keller'],
    '2023-01': [567, 516, 691, 804],
    '2023-02': [506, 527, 639, 766],
    '2023-03': [645, 720, 852, 899],
    '2023-04': [568, 638, 771, 822]
}

new_df = pd.DataFrame(data)
# Melt the dataframe to convert from wide format to long format
melted_df = pd.melt(new_df, id_vars=['Location'], var_name='servicePeriod', value_name='Amount')

# Rename columns to match the original dataframe
melted_df = melted_df.rename(columns={'Location': 'locationName'})
melted_df['Quarter'] = pd.to_datetime(melted_df['servicePeriod']).dt.to_period('Q')

In [ ]:
"""Total Encounters"""
sortMatrix = {'Arlington': 1, 'Denton': 2, 'Frisco': 3, 'Keller': 4, 'Weatherford': 5, 'Las Colinas': 6, 'Tyler': 7, 'Sherman':8 }

EncounterCounts = df.groupby(['locationName','Quarter'],as_index=False)['Amount'].sum()
# Concatenate with the original dataframe\
EncounterCountsGraph = pd.concat([EncounterCounts, melted_df], ignore_index=True)
# EncounterCountsGraph = EncounterCounts.copy()
EncounterCountsGraph['sortByArray'] = EncounterCountsGraph['locationName'].map(sortMatrix)
EncounterCountsGraph = EncounterCountsGraph.groupby(['locationName','Quarter'],as_index=False)['Amount'].sum()

# EncounterCountsGraph['Quarter'] = pd.to_datetime(EncounterCountsGraph['servicePeriod']).dt.to_period('Q')
EncounterCountsGraph.head()

,locationName,Quarter,Amount
0,Arlington,2023Q1,1721.0
1,Arlington,2023Q2,1949.0
2,Arlington,2023Q3,1966.0
3,Arlington,2023Q4,1835.0
4,Arlington,2024Q1,1700.0


In [ ]:
"""New Patients"""
new_pts = df[(df['appointmentType'].str.lower().str.contains('new')) & (df['Quarter'] >= '2023Q2')]\
    .groupby(['locationName','Quarter','appointmentType'],as_index=False)['patientChartNumber'].count()\
    .rename(columns={'patientChartNumber':'Amount'}).sort_values(by=['locationName','Quarter'])
new_pts.head(2)   

,locationName,Quarter,appointmentType,Amount
0,Arlington,2023Q2,NEW PATIENT,14
1,Arlington,2023Q3,NEW PATIENT,27


In [ ]:
"""Total Patients"""
total_pts = df[(df['Quarter'] >= '2023Q2')]\
    .groupby(['Quarter','patientChartNumber'],as_index=False)['appointmentType'].count()\
    .merge(df, on='patientChartNumber', how='left').drop_duplicates(subset=['patientChartNumber','Quarter_x'])\
    [['Quarter_x','locationName','patientChartNumber']]   

total_pts = total_pts.groupby(['locationName','Quarter_x'],as_index=False)['patientChartNumber'].count()\
.rename(columns={'Quarter_x':'Quarter','patientChartNumber':'Total Patients'})

total_pts.head(2)

,locationName,Quarter,Total Patients
0,Arlington,2023Q2,93
1,Arlington,2023Q3,106


In [ ]:
df['Year'] = pd.to_datetime(df['servicePeriod']).dt.to_period('Y')

In [ ]:
year_to_date_total = df[(df['Quarter'] >= '2023Q2')]\
    .groupby(['Year','patientChartNumber'],as_index=False)['appointmentType'].count()\
    .merge(df, on='patientChartNumber', how='left').drop_duplicates(subset=['patientChartNumber','Year_x'])\
    [['Year_x','locationName','patientChartNumber']]\
    .groupby(['locationName','Year_x'],as_index=False)['patientChartNumber'].count()\
    .rename(columns={'Year_x':'Quarter','patientChartNumber':'Amount'})   

year_to_date_total['Type'] = 'TotalPatients'

In [ ]:
"""Compliance"""
df['serviceDate'] = pd.to_datetime(df['serviceDate'])

df1 = df[(df['Quarter'] >= '2023Q2')]\
    .groupby(['locationName','serviceDate','patientChartNumber','Quarter'],as_index=False)['appointmentType'].count()

df1['Week'] = df1['serviceDate'].dt.strftime('%U')

df2 = df1.groupby(['locationName','Week','Quarter','patientChartNumber'],as_index=False)['appointmentType'].sum()\
    .groupby(['locationName','Quarter'],as_index=False)['appointmentType'].mean().round(2) 

df2.head(2)

,locationName,Quarter,appointmentType
0,Arlington,2023Q2,2.07
1,Arlington,2023Q3,2.14


In [ ]:
"""Lost Patients"""

# Find new patients for all periods
all_periods = sorted(df['Quarter'].unique())
lost_patients_all_periods = {}

for i in range(1, len(all_periods)):

    current_period = all_periods[i]
    previous_period = all_periods[i - 1]

    # print(f'Current period: {current_period} - Previous period: {previous_period}')

    current_patients = df[df['Quarter'] == current_period]['patientChartNumber'].unique()
    previous_patients = df[df['Quarter'] == previous_period]['patientChartNumber'].unique()

    lost_patients = set(previous_patients) - set(current_patients)
    

    lost_patients = set(previous_patients) - set(current_patients)
    lost_patients_all_periods[current_period] = lost_patients


In [ ]:
lp_df = pd.DataFrame([
    {'Quarter': period, 'patientChartNumber': patient}
    for period, patients in lost_patients_all_periods.items()
    for patient in patients
])

lp_df['count'] = 1

In [ ]:
lp_df = lp_df.groupby(['Quarter','patientChartNumber'],as_index=False)['count'].count()\
      .merge(df, on='patientChartNumber', how='left')[['Quarter_x','locationName','patientChartNumber','count']]\
      .drop_duplicates(subset=['Quarter_x','patientChartNumber'])\
      .groupby(['locationName','Quarter_x'],as_index=False)['count'].sum()

In [ ]:

# Rename columns to have the same name and add a new column to indicate the type
new_pts = new_pts.rename(columns={'appointmentType': 'Type', 'Amount': 'Amount'})
total_pts = total_pts.rename(columns={'Total Patients': 'Amount'})
df2 = df2.rename(columns={'appointmentType': 'Amount'})
lp_df = lp_df.rename(columns={'Quarter_x': 'Quarter', 'count': 'Amount'})
EncounterCountsGraph = EncounterCountsGraph.rename(columns={'Amount': 'Amount'})


# Add a new column to indicate the type of data
new_pts['Type'] = 'NewPatients'
total_pts['Type'] = 'TotalPatients'
df2['Type'] = 'Compliance'
lp_df['Type'] = 'LostPatients'
EncounterCountsGraph['Type'] = 'Encounters'

# Concatenate the dataframes
combined_df = pd.concat(
    [new_pts[['locationName', 'Quarter', 'Amount', 'Type']],
     total_pts[['locationName', 'Quarter', 'Amount', 'Type']],
     df2[['locationName', 'Quarter', 'Amount', 'Type']],
     lp_df[['locationName', 'Quarter', 'Amount', 'Type']],
     EncounterCountsGraph[['locationName', 'Quarter', 'Amount', 'Type']]],
    ignore_index=True
)

# combined_df['Quarter'] = pd.to_datetime(combined_df['servicePeriod'] + '-01').dt.to_period('Q').astype(str)
combined_df.head(2)

,locationName,Quarter,Amount,Type
0,Arlington,2023Q2,14.0,NewPatients
1,Arlington,2023Q3,27.0,NewPatients


In [ ]:
location_updates = {
       'Las Colinas': '2023Q4',
       'Weatherford': '2023Q4',
       'Sherman': '2024Q3',
       'Tyler': '2024Q1'
}

for location, period in location_updates.items():
       combined_df.loc[(combined_df['locationName'] == location) & (combined_df['Quarter'] < period), 'locationName'] = 'Keller'


In [ ]:
year_to_date_total = year_to_date_total.rename(columns={'patientChartNumber': 'Amount'})
combined_df_final = pd.concat([combined_df, year_to_date_total[['locationName', 'Quarter', 'Amount', 'Type']]])

combined_df_final.head(2)

,locationName,Quarter,Amount,Type
0,Arlington,2023Q2,14.0,NewPatients
1,Arlington,2023Q3,27.0,NewPatients
2,Arlington,2023Q4,28.0,NewPatients
3,Arlington,2024Q1,21.0,NewPatients
4,Arlington,2024Q2,34.0,NewPatients
...,...,...,...,...
10,Sherman,2023,1.0,TotalPatients
11,Sherman,2024,25.0,TotalPatients
12,Tyler,2024,84.0,TotalPatients
13,Weatherford,2023,37.0,TotalPatients


In [ ]:
combined_df_final['Quarter'] = combined_df_final['Quarter'].astype(str)
combined_df_final['Update Time'] = pd.to_datetime('now',utc=True).strftime('%Y-%m-%d %H:%M:%S')
combined_df_final.rename(columns={'Quarter': 'servicePeriod'}, inplace=True)

In [ ]:
combined_df_final.to_sql('ScoreCardsQuarterly', con=engine, if_exists='replace', index=False)

241